In [1]:
import numpy as np 
import pandas as pd 
import json 


!pip install folium==0.5.0
import folium 
from folium import plugins

from geopy.geocoders import Nominatim #convert an address into latitude and longitude values

import requests #library to handle requests
from pandas.io.json import json_normalize #tranform JSON file into a pandas dataframe

#set up the plotting tools
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

import seaborn as sns

#import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist

print('Libraries imported.')

Libraries imported.


In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)  

In [4]:
  neighborhoods_data = newyork_data['features']
    
    
    # define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [5]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [6]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [7]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
print(len(manhattan_data))
manhattan_data.head()

40


,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [8]:
# create map of NYC using latitude and longitude values
map_nyc = folium.Map(location=[latitude, longitude], zoom_start=10)
print(type(neighborhoods))
print(neighborhoods)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_nyc)  
    
map_nyc

<class 'pandas.core.frame.DataFrame'>
           Borough         Neighborhood   Latitude  Longitude
0            Bronx            Wakefield  40.894705 -73.847201
1            Bronx           Co-op City  40.874294 -73.829939
2            Bronx          Eastchester  40.887556 -73.827806
3            Bronx            Fieldston  40.895437 -73.905643
4            Bronx            Riverdale  40.890834 -73.912585
5            Bronx          Kingsbridge  40.881687 -73.902818
6        Manhattan          Marble Hill  40.876551 -73.910660
7            Bronx             Woodlawn  40.898273 -73.867315
8            Bronx              Norwood  40.877224 -73.879391
9            Bronx       Williamsbridge  40.881039 -73.857446
10           Bronx           Baychester  40.866858 -73.835798
11           Bronx       Pelham Parkway  40.857413 -73.854756
12           Bronx          City Island  40.847247 -73.786488
13           Bronx         Bedford Park  40.870185 -73.885512
14           Bronx   University 

In [9]:
CLIENT_ID = 'OLOVAGZTHZNUELBTPLCRZJLJZA134SAJYXHHWQXX5PJVYFLG' # your Foursquare ID
CLIENT_SECRET = 'DQVHJMI1IZCASIGQTUSE1EAUAG4LHS2OYKM2XRR04HK0NDZU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OLOVAGZTHZNUELBTPLCRZJLJZA134SAJYXHHWQXX5PJVYFLG
CLIENT_SECRET:DQVHJMI1IZCASIGQTUSE1EAUAG4LHS2OYKM2XRR04HK0NDZU


In [10]:
#Get each neighborhood name and lat / long values 

manhattan_data.loc[0, 'Neighborhood']

latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               latitude, 
                                                               longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


In [11]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 600 # define radius
categoryId = '4d4ae6fc7a7b7dea34424761'
# create URL
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT,
    categoryId
)
url # display URL

'https://api.foursquare.com/v2/venues/search?&client_id=OLOVAGZTHZNUELBTPLCRZJLJZA134SAJYXHHWQXX5PJVYFLG&client_secret=DQVHJMI1IZCASIGQTUSE1EAUAG4LHS2OYKM2XRR04HK0NDZU&v=20180605&ll=40.87655077879964,-73.91065965862981&radius=600&limit=100&categoryId=4d4ae6fc7a7b7dea34424761'

In [12]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d21543cc0c08400257cfac5'},
 'response': {'venues': [{'id': '4eb9cdd55c5c0e067d1488c4',
    'name': 'Popeyes Louisiana Kitchen',
    'location': {'address': '205 W 231st St',
     'crossStreet': 'at Broadway',
     'lat': 40.879311888822194,
     'lng': -73.90479758629137,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.879311888822194,
       'lng': -73.90479758629137}],
     'distance': 581,
     'postalCode': '10463',
     'cc': 'US',
     'city': 'Bronx',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['205 W 231st St (at Broadway)',
      'Bronx, NY 10463',
      'United States']},
    'categories': [{'id': '4d4ae6fc7a7b7dea34424761',
      'name': 'Fried Chicken Joint',
      'pluralName': 'Fried Chicken Joints',
      'shortName': 'Fried Chicken',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/friedchicken_',
       'suffix': '.png'},
      'primary': True}],
    'referralId'

In [13]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
venues = results['response']['venues']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,categories,hasPerk,id,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,name,referralId,categories
0,"[{'id': '4d4ae6fc7a7b7dea34424761', 'name': 'F...",False,4eb9cdd55c5c0e067d1488c4,205 W 231st St,US,Bronx,United States,at Broadway,581,"[205 W 231st St (at Broadway), Bronx, NY 10463...","[{'label': 'display', 'lat': 40.87931188882219...",40.879312,-73.904798,10463,NY,Popeyes Louisiana Kitchen,v-1562465340,Fried Chicken Joint


In [15]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=1600):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    lng, 
    radius, 
    LIMIT,
    categoryId
        )
        print(url)
            
        # make the GET request
        results = requests.get(url).json()['response']['venues']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude'
                            ]
    
    return(nearby_venues)

In [17]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                                                     )

Marble Hill
https://api.foursquare.com/v2/venues/search?&client_id=OLOVAGZTHZNUELBTPLCRZJLJZA134SAJYXHHWQXX5PJVYFLG&client_secret=DQVHJMI1IZCASIGQTUSE1EAUAG4LHS2OYKM2XRR04HK0NDZU&v=20180605&ll=40.87655077879964,-73.91065965862981&radius=1600&limit=100&categoryId=4d4ae6fc7a7b7dea34424761
Chinatown
https://api.foursquare.com/v2/venues/search?&client_id=OLOVAGZTHZNUELBTPLCRZJLJZA134SAJYXHHWQXX5PJVYFLG&client_secret=DQVHJMI1IZCASIGQTUSE1EAUAG4LHS2OYKM2XRR04HK0NDZU&v=20180605&ll=40.71561842231432,-73.99427936255978&radius=1600&limit=100&categoryId=4d4ae6fc7a7b7dea34424761
Washington Heights
https://api.foursquare.com/v2/venues/search?&client_id=OLOVAGZTHZNUELBTPLCRZJLJZA134SAJYXHHWQXX5PJVYFLG&client_secret=DQVHJMI1IZCASIGQTUSE1EAUAG4LHS2OYKM2XRR04HK0NDZU&v=20180605&ll=40.85190252555305,-73.93690027985234&radius=1600&limit=100&categoryId=4d4ae6fc7a7b7dea34424761
Inwood
https://api.foursquare.com/v2/venues/search?&client_id=OLOVAGZTHZNUELBTPLCRZJLJZA134SAJYXHHWQXX5PJVYFLG&client_secret=DQVHJM

In [18]:
len(manhattan_data['Neighborhood'])

40

In [37]:
print(manhattan_venues.shape)
manhattan_venues.groupby('Neighborhood').count().sort_values(by='Neighborhood Latitude', ascending=False)



(1106, 6)


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Neighborhood,,,,,
Midtown South,49,49,49,49,49
Flatiron,47,47,47,47,47
Central Harlem,46,46,46,46,46
Midtown,44,44,44,44,44
Manhattanville,41,41,41,41,41
Murray Hill,41,41,41,41,41
Gramercy,41,41,41,41,41
Little Italy,38,38,38,38,38
Greenwich Village,37,37,37,37,37


In [40]:
manhattan_venues.loc[manhattan_venues['Neighborhood']== 'Midtown South']

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
884,Midtown South,40.74851,-73.988713,bb.q chicken,40.747863,-73.986965
885,Midtown South,40.74851,-73.988713,Turntable LP Bar & Karaoke,40.747974,-73.987239
886,Midtown South,40.74851,-73.988713,Turntable Chicken Jazz,40.748428,-73.986302
887,Midtown South,40.74851,-73.988713,Pelicana Chicken,40.747602,-73.986272
888,Midtown South,40.74851,-73.988713,Sticky’s Finger Joint,40.754780,-73.986520
889,Midtown South,40.74851,-73.988713,Advil Bottle,40.761390,-73.975950
890,Midtown South,40.74851,-73.988713,Fleurimond Catering,40.760194,-73.996699
891,Midtown South,40.74851,-73.988713,KFC,40.750041,-73.992211
892,Midtown South,40.74851,-73.988713,Chickls,40.750242,-73.987390
893,Midtown South,40.74851,-73.988713,Chick’ncone,40.754860,-73.986659


Map of Manhattan neighborhoods with top 10 clustered venues

In [21]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue']], prefix="", prefix_sep="")


# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

print(manhattan_onehot.shape)
manhattan_onehot.head()

(1106, 119)


,Neighborhood,#BlazinFeathers,Advil Bottle,Amy Ruth's,Atomic Wings,Banzai Phresh,Best Wingers,Big Kennedy Fried Chicken,Big NY Fried Chicken,"Birdies, Grandma's Chicken For The People",...,U.S. Chicken,Unidentified Flying Chicken Hell's Kitchen,United Fried Chicken,Us Fried Chicken & Pizza,Village Fried Chicken,Wonder Fried Chicken & Pizza,bb.q chicken,kennedy chicken & pizza,roast,the guild art gallery
0,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
print(manhattan_grouped.shape)
manhattan_grouped

(40, 119)


,Neighborhood,#BlazinFeathers,Advil Bottle,Amy Ruth's,Atomic Wings,Banzai Phresh,Best Wingers,Big Kennedy Fried Chicken,Big NY Fried Chicken,"Birdies, Grandma's Chicken For The People",...,U.S. Chicken,Unidentified Flying Chicken Hell's Kitchen,United Fried Chicken,Us Fried Chicken & Pizza,Village Fried Chicken,Wonder Fried Chicken & Pizza,bb.q chicken,kennedy chicken & pizza,roast,the guild art gallery
0,Battery Park City,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Carnegie Hill,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.058824,0.000000
2,Central Harlem,0.000000,0.000000,0.021739,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.021739,0.021739,0.021739,0.000000,0.000000,0.000000,0.000000,0.000000
3,Chelsea,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.027778,0.000000,0.000000,0.000000,0.000000,0.027778,0.000000,0.000000,0.027778
4,Chinatown,0.031250,0.000000,0.000000,0.000000,0.000000,0.031250,0.000000,0.000000,0.031250,...,0.031250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Civic Center,0.000000,0.000000,0.000000,0.000000,0.000000,0.037037,0.000000,0.000000,0.000000,...,0.037037,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Clinton,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.031250,0.000000,0.000000,0.000000,0.000000,0.031250,0.000000,0.000000,0.000000
7,East Harlem,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000
8,East Village,0.030303,0.000000,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.030303,...,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030303
9,Financial District,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [23]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
                       venue  freq
0  Popeyes Louisiana Kitchen  0.12
1                        KFC  0.12
2                       Fuku  0.12
3          Cornerstone Grill  0.06
4                Chick-fil-A  0.06


----Carnegie Hill----
                           venue  freq
0  Kennedy Fried Chicken & Pizza  0.12
1                            KFC  0.12
2          Kennedy Fried Chicken  0.06
3      Texas Chicken and Burgers  0.06
4      JFK Fried Chicken & Pizza  0.06


----Central Harlem----
                       venue  freq
0        Crown Fried Chicken  0.17
1  Popeyes Louisiana Kitchen  0.15
2      Kennedy Fried Chicken  0.13
3                        KFC  0.11
4    Texas Chicken & Burgers  0.04


----Chelsea----
                       venue  freq
0  Popeyes Louisiana Kitchen  0.11
1                        KFC  0.06
2           Pelicana Chicken  0.06
3      Sticky's Finger Joint  0.06
4                Chick’ncone  0.03


----Chinatown----
                       

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,KFC,Fuku,Popeyes Louisiana Kitchen,Dirty Bird To Go,Sticky's Finger Joint,Captain's Fried Chicken,Kentucky Fried Lizard,Thunder Bun,Dees Chicken & Waffles,Mad For Chicken
1,Carnegie Hill,KFC,Kennedy Fried Chicken & Pizza,Geteburgers,Chirping Chicken,Popeyes Louisiana Kitchen,PortuGrill NYC,Roast Homestyle Chicken,Crown Fried Chicken,Texas Chicken and Burgers,Fried Chicken Shack
2,Central Harlem,Crown Fried Chicken,Popeyes Louisiana Kitchen,Kennedy Fried Chicken,KFC,Texas Chicken & Burgers,Kennedy Fried Chicken & Pizza,Kings Fried Chicken,Charles' Pan Fried Chicken,Domino's Pizza,Kennedy's Fried Chicken
3,Chelsea,Popeyes Louisiana Kitchen,Pelicana Chicken,KFC,Sticky's Finger Joint,Fuku,NYC Fried Chicken Corporation,Bonchon Chicken,Bugaboo,K Chicken,Chick-fil-A
4,Chinatown,KFC,Fuku,Popeyes Louisiana Kitchen,Chick-fil-A,#BlazinFeathers,Chicken V,Royal Fried Chicken,Root & Bone,Captain's Fried Chicken,Pudgie's Famous Chicken


In [26]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 0, 3, 1, 1, 3, 0, 1, 1], dtype=int32)

In [27]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,0,Popeyes Louisiana Kitchen,KFC,Kennedy Fried Chicken,Kennedy Fried Chicken & Pizza,Texas Chicken & Burgers,Texas Chicken and Burgers,"Kennedy's Fried Chicken, Pizza & Burgers",Kennedy's Fried Chicken,Fordham Fried Chicken & Seafood Inc,Kennedy Chicken Grill & Gyro
1,Manhattan,Chinatown,40.715618,-73.994279,1,KFC,Fuku,Popeyes Louisiana Kitchen,Chick-fil-A,#BlazinFeathers,Chicken V,Royal Fried Chicken,Root & Bone,Captain's Fried Chicken,Pudgie's Famous Chicken
2,Manhattan,Washington Heights,40.851903,-73.936900,0,Kennedy Fried Chicken,Popeyes Louisiana Kitchen,Kennedy Fried Chicken & Pizza,Crown Fried Chicken,Kennedy Chicken & Gyro,KFC,Big Kennedy Fried Chicken,Kennedy Fried Chicken - Nagle,Sto Domingo Grocery,Kennedy's Chicken And Grill
3,Manhattan,Inwood,40.867684,-73.921210,0,Kennedy Fried Chicken & Pizza,Popeyes Louisiana Kitchen,KFC,Kennedy Fried Chicken,"Kennedy's Fried Chicken, Pizza & Burgers",Crown Fried Chicken,Kennedy fried chicken,Kennedy Fried Chicken And Pizza,Kennedy Fried Chicken - Nagle,Big Kennedy Fried Chicken
4,Manhattan,Hamilton Heights,40.823604,-73.949688,0,Crown Fried Chicken,Kennedy Fried Chicken,KFC,Popeyes Louisiana Kitchen,Texas Chicken & Burgers,Kennedy's Chicken And Grill,Domino's Pizza,Krispy Krunchy Chicken,Boston Market,Kings Fried Chicken
5,Manhattan,Manhattanville,40.816934,-73.957385,0,Crown Fried Chicken,Kennedy Fried Chicken,Popeyes Louisiana Kitchen,KFC,Texas Chicken & Burgers,Kings Fried Chicken,Domino's Pizza,Kennedy's Fried Chicken,Charles' Pan Fried Chicken,Chirping Chicken
6,Manhattan,Central Harlem,40.815976,-73.943211,0,Crown Fried Chicken,Popeyes Louisiana Kitchen,Kennedy Fried Chicken,KFC,Texas Chicken & Burgers,Kennedy Fried Chicken & Pizza,Kings Fried Chicken,Charles' Pan Fried Chicken,Domino's Pizza,Kennedy's Fried Chicken
7,Manhattan,East Harlem,40.792249,-73.944182,0,Popeyes Louisiana Kitchen,Crown Fried Chicken,Kennedy Fried Chicken,Kennedy Fried Chicken & Pizza,KFC,JFK Fried Chicken & Pizza,Texas Chicken and Burgers,Melba's American Comfort Food,Mama's Fried Chicken,Corner Chicken & Waffles
8,Manhattan,Upper East Side,40.775639,-73.960508,4,Chirping Chicken,PortuGrill NYC,Broadway Chicken,Leslie's Fried Chicken,Chick-fil-A,Geteburgers,Roast Homestyle Chicken,Chick’nCone,Kennedy Fried Chicken & Pizza,Atomic Wings
9,Manhattan,Yorkville,40.775930,-73.947118,2,Popeyes Louisiana Kitchen,Kennedy Fried Chicken & Pizza,Roast Homestyle Chicken,Palace Fried Chicken,Texas Chicken and Burgers,Fried Chicken Shack,PortuGrill NYC,CHUBBY BURGERS chicken and pizza,KFC,Chirping Chicken


In [28]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
colors_array = cm.rainbow(np.linspace(0, 1, 10))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)

# add markers to the map
markers_colors = []

for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
    
 
    
       
map_clusters

['#8000ff', '#4856fb', '#10a2f0', '#2adddd', '#62fbc4', '#9cfba4', '#d4dd80', '#ffa256', '#ff562c', '#ff0000']
